In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima.model import ARIMA

In [7]:

df = pd.read_csv("../../data/cleaned_data.csv")
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
# InvoiceDate'i datetime formatına çeviriyoruz
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

# Ay bazında bir sütun oluşturuyoruz
df['YearMonth'] = df['InvoiceDate'].dt.to_period('M')

# Her ürün için ay ay toplam satış miktarını hesaplıyoruz
monthly_sales = df.groupby(['Description', 'YearMonth'])['Quantity'].sum().reset_index()

# Period objesini stringe çevir (Prophet kullanacaksan datetime gerekiyor)
monthly_sales['YearMonth'] = monthly_sales['YearMonth'].astype(str)

monthly_sales.head()

,Description,YearMonth,Quantity
0,4 PURPLE FLOCK DINNER CANDLES,2022-12,14
1,4 PURPLE FLOCK DINNER CANDLES,2023-01,1
2,4 PURPLE FLOCK DINNER CANDLES,2023-02,1
3,4 PURPLE FLOCK DINNER CANDLES,2023-04,7
4,4 PURPLE FLOCK DINNER CANDLES,2023-05,1


In [4]:
from prophet import Prophet

# Örnek: Belirli bir ürün seçelim
product = 'WHITE METAL LANTERN'
product_sales = monthly_sales[monthly_sales['Description'] == product]

# Prophet formatına getir
df_prophet = product_sales.rename(columns={'YearMonth': 'ds', 'Quantity': 'y'})
df_prophet['ds'] = pd.to_datetime(df_prophet['ds'])

# Modeli eğit
model = Prophet()
model.fit(df_prophet)

# Gelecek 1 ayı tahmin et
future = model.make_future_dataframe(periods=1, freq='M')
forecast = model.predict(future)

# Sadece tahmin edilen son ayı görelim
forecast[['ds', 'yhat']].tail(1)


12:25:55 - cmdstanpy - INFO - Chain [1] start processing
12:25:56 - cmdstanpy - INFO - Chain [1] done processing


,ds,yhat
12,2023-11-30,63.926824
